# Data, in Python, with THREE Dimensions!

Hi!  My name's Matt.  I'll be working through some things in python with you that I think are cool, and that relate to visualizing things in three dimensions.

## Libraries We'll Use

We're going to use a handful of libraries, as well as a couple widget libraries for Jupyter.  You'll need:

 * numpy
 * matplotlib
 * plotly
 * yt (from git)
 * ipyvolume
 * pythreejs
 
There are other tools that might be helpful!  But we won't be touching on them.  I'm thinking here specifically of `xarray`, `dask` and a couple others.  We'll start by importing a couple of these, and then as time goes on we'll add more and more.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## What makes 3D data special?

 > 3D data is special -- it's just like 2D data, but *more*.

We're mostly going to talk about data that has some degree of spatial organization in three dimensions.  We will return to how to look at data that does not have a spatial organization in all three, but for now, let's start with that.

The key thing here is that we have a mechanism of measuring the *distance* between the first three columns ($x, y, z$).  In fact, we know that the distance can be computed:

$ \sqrt{(x_1 - x_0)^2 + (y_1 - y_0)^2 + (z_1 - z_0)^2}$

This is really cool!  It means that -- built in! -- you already have an idea of how to represent these points, and how to measure the distance between them.  Not only that, it also means that there's a prescription for how to visualize the points from a certain viewpoint.

## Data!  And its storage ...

We're going to talk about basically *two* different ways of storing data for 3D systems.  There are more, and I'll allude to them a little bit, but let's start with the two you're the most likely to see and deal with.

### Discrete

Discretely sampled 3D data is typically ordered as columns (but, this is not always the case!) where three of the columns can be assigned to dimensions corresponding to spatial positions -- for instance, $x, y, z$ or maybe $r, \theta, \phi$.  It should also be noted that in many cases, you'll also be presented with a column that corresponds to some form of a "radius of influence," or a set of columns that describe influences along different directions (either axially aligned, or axes themsevles.)

So for instance, you might have a situation where your data comes to you looking something like this:

| x | y | z | temperature |
|-|-|-|-|
| 0.5 | 0.5 | 0.5 | 100 |
| 0.25 | 0.1 | 0.4 | 150 |
| 0.71 | 0.9 | 0.9 | 200 |

In this particular case, you now have three rows, where the x, y, z variables and a field variable are defined.  Now, usually, you'll have lots *more* data -- this is just a pretty simple set, because I was trying to come up with a set of random values!

These types of data typically come in one of a handful of ways.  You're most likely to see them structured as some form of either multiple numpy arrays or a single numpy array with a complex dtype or exactly two dimensions (where one dimension is the axis and the other is the element.)  We can create these in numpy like this:

In [ ]:
arr = np.array([ [0.5, 0.5, 0.5, 100],
                 [0.25, 0.1, 0.4, 150],
                 [0.71, 0.9, 0.9, 200]])
print(arr.shape)

This is the simplest way of representing it.  Looking at it, you'll see that since we've organized it in the way we did, it comes out to a shape of `(3, 4)`, which indicates that our first axis is the row and the second is the column.  We don't know the names, so that makes it a bit tricky unless we knew the names of the columns beforehand!

Let's try a different way.  This way we will use a `dtype` in numpy.

In [ ]:
arr_dtype = np.array([ (0.5, 0.5, 0.5, 100),
                       (0.25, 0.1, 0.4, 150),
                       (0.71, 0.9, 0.9, 200)],
                    dtype = [ ('x', 'f8'), ('y', 'f8'), ('z', 'f8'), ('T', 'f8') ] )

In [ ]:
print(arr_dtype)
print(arr_dtype.shape)

This is different!  And for a couple reasons, too.  Note that in our specification, we've named our fields `x`, `y`, `z` and `T`.  We've also changed our values that we feed in to `np.array` to be a list of tuples, rather than a list of lists.  (Try it with a list of lists, and see if you can figure out what it does!)

And, finally, when we look at the shape, it says it's of shape `(3,)`!  This is different because it tells us the number of items of the given `dtype` -- not the number of `f8` values, for instance, but the number of collections of `x`, `y`, `z`, and `T`!  We can see this in action by looking at `arr_dtype.dtype.itemsize`.

Now we're able to access by-field:

In [ ]:
arr_dtype['x']

Now, in general, this isn't necessarily the best way to access data.  But, if you're using numpy to simply load data off disk, and that data is organized in this way, it does pose some advantages, *especially* if it's a mix of different types -- for instance, if you have an `ID` field that's an integer.

The final way you might see things stored in memory using just numpy -- and in some ways, this is similar to how pandas provides access to data -- is through a dictionary of arrays.

In [ ]:
arr_dict = dict(
    x = np.array([0.5, 0.25, 0.71]),
    y = np.array([0.25, 0.71, 0.4]),
    z = np.array([0.71, 0.9, 0.9]),
    T = np.array([100, 150, 200])
)

Here, we can access them like you might expect, but it's harder to do row-based reductions.  And yet, this is the way you'll probably see it most of the time.

### Regular

Other times, you will find that the data you have access to comes in regularly shaped, multi-dimensional arrays.

Whereas before you were usually using some form of a set of columns, in this case you'll be using either one or multiple arrays with three dimensions.

In most of these cases, the data points will be assumed to be spaced out evenly; in some cases, however, you might find that there is a supplemental array for each dimension that describes the distance between cell values.

**Note:** It's important when dealing with data provided in 3D arrays like this to recognize whether the data comes from a regular sampling (and thus represents sample points) or if it is assumed to be part of a volumetric average over the volume of each individual element.

We can create some fake data in this way with numpy functions; for instance, let's create a field that is $1/r^2$ from the point (-0.05, -0.05, -0.05).

In [ ]:
center = [-0.05, -0.05, -0.05]
x = np.mgrid[0.0:1.0:128j] # 128 (inclusive) steps from 0.0 to 1.0
y = np.mgrid[0.0:1.0:128j]
z = np.mgrid[0.0:1.0:128j]

We'll use broadcasting tricks to compute the radius field in 3D, and to end up with an array of shape (128, 128, 128).  If you haven't seen these before, the end result is to insert "fake" axes into our arrays, to encourage numpy to "broadcast" them out and fill the dimensions.  When we use a `None` value as a slice, it will implicitly expand the array in that dimension.

In [ ]:
ir2 = (((x - center[0])**2)[:, None, None]
      +((y - center[1])**2)[None, :, None]
      +((z - center[2])**2)[None, None, :])**-1
print(ir2.shape)

Note that you can imagine having record or dtype-ful arrays in this situation too, or you might be in a situation where there's either an extra axis at the beginning or at the end for the "field" name.

When dealing with data in 3D, there's one additional thing to be aware of, particularly when the data is really big -- which data values are next to which.  This won't *always* be important, but if you're using really big datasets, it can have a non-negligible impact on the speed with which you're manipulating data.  We can see that by looking at the ordering of the array.  If it's C-ordered, then `arr[0,0,0]` is right next to `arr[0,0,1]` in memory.  If it's Fortran ordered, it'll be the opposite -- `arr[0,0,0]` will be right next to `arr[1,0,0]`.  (There are other ways to "stride" data, but these are the two most common.)

You might also find yourself with data that *isn't* evenly spread out.  We can (with a little effort!) generate some, of the same type as before.

In [ ]:
x = np.add.accumulate(np.random.random(128))
x /= x.max()

y = np.add.accumulate(np.random.random(128))
y /= y.max()

z = np.add.accumulate(np.random.random(128))
z /= z.max()

ir2 = (((x - center[0])**2)[:, None, None]
      +((y - center[1])**2)[None, :, None]
      +((z - center[2])**2)[None, None, :])**-1
print(ir2.shape)

### Other...?

There are a few other ways that you might find 3D data stored, though!  We won't really go into them, but a few you might see are storing data at the vertices, edges, or other nodal locations on unstructured meshes, as well as potentially regular structures defined in ways that look like discrete sampling.